In [8]:
import sys
sys.path.append('C:\AI\AI_Project_2')
from referee.game import Board, SpawnAction, HexPos, SpreadAction, HexDir,Action
from boardupdate import spawn_board, spread_board, render_board, apply_ansi, can_spawn
import random




def get_illegal_spread(turn:str, state: dict[tuple, tuple]):
    spread_list = []
    direction_list = [(1,-1),(-1,1),(1,0),(0,1),(0,-1),(-1,0)]
    length = len(direction_list)
    if turn == 'r':
        for (r,q) in state:
            if state[(r,q)][0] == 'r':
                for i in range(length):
                    t = direction_list[i]
                    spread_list.append((r,q,t[0],t[1]))
        turn = 'b'
    else:
        for (r,q) in state:
            if state[(r,q)][0] == 'b': 
                for i in range(length):
                    t = direction_list[i]
                    spread_list.append((r,q,t[0],t[1]))
        turn = 'b'
    return spread_list

class Game:
    def __init__(self):
        self.state = {}
        self.turn = 'r'
        self.action_count = 0
    def count_power(self,color: str) -> int:
        power = 0
        for value in self.state.values():
            if value[0] == color:
                power += value[1]
        return power
    




def next_turn(turn:str)->str:
    if turn == 'r':
        return 'b'
    else:
        return 'r'
    


def take_action(game:Game):
    
    action_list = []
    spread_list = []
    spawn_list = []
    spread_list = get_illegal_spread(game.turn, game.state)
    spawn_list = can_spawn(game.state)
    action_list = spread_list + spawn_list
    upperbound = len(action_list) - 1
    lowerbound = 0
    random_index = random.randint(lowerbound, upperbound)
    if random_index >= len(spread_list):
        spawn = action_list[random_index]
        game.state = spawn_board(game.state, spawn, game.turn)
    else:
        spread = action_list[random_index]
        game.state = spread_board(game.state, spread, game.turn)
    game.action_count +=1

def end(game: Game)-> int:
    continue_game = 100
    if game.action_count == 343:
        r_power = game.count_power('r')
        b_power = game.count_power('b')
        if r_power - b_power >= 2:
            return 1
        elif b_power - r_power >= 2:
            return -1
        else:
            return 0
        
    if game.action_count > 1:
        if game.turn == 'r':
            for value in game.state.values():
                if value[0] == 'b':
                    return continue_game
            return 1

        if game.turn == 'b':
            for value in game.state.values():
                if value[0] == 'r':
                    return continue_game
            return -1
    return continue_game



game = Game()

def simulation(game: Game):
    for i in range(343):
        take_action(game)
        #print(f"The current turn is: {game.turn} ")
        if end(game) == 100:
            game.turn = next_turn(game.turn)
        else:
            if end(game) == 1:
                print("red wins!")
            elif end(game) == -1:
                print("blue wins!")
            else:
                print("draw!")
            print(game.action_count)
            print(game.count_power('r')+ game.count_power('b'))
            print(render_board(game.state))
            break



def reset_game(game: Game):
    game.state = {}
    game.action_count = 0
    game.turn = 'r'
    
for i in range(5):
    simulation(game)
    reset_game(game)




red wins!
343
49
                         ..     
                     r1      ..     
                 r5      b2      r2     
             ..      b1      ..      r1     
         r1      ..      ..      ..      r5     
     ..      ..      ..      r1      b1      r1     
 b1      r1      b2      ..      b2      b6      ..     
     ..      b2      ..      b1      ..      r1     
         b2      r2      b1      ..      ..     
             ..      ..      ..      r1     
                 r3      ..      r1     
                     r1      r1     
                         ..     

blue wins!
343
49
                         ..     
                     ..      ..     
                 b2      b2      ..     
             ..      ..      b2      r3     
         ..      r2      ..      ..      ..     
     b1      ..      b3      ..      ..      b2     
 ..      ..      b6      ..      b2      r1      r1     
     b1      r1      ..      r1      r1      ..     
         ..      b1    